## Load Libraries

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
sys.path.append("../")
# import geopy
from geopy import distance
import math
import os

### Locations and Distances for government sensors

In [2]:
# location_df = pd.read_csv("/scratch/ab9738/pollution_img/govdata/locations.csv")

# location_df = location_df.drop(["source"], axis=1)

# columns = [loc for loc in location_df['id']]

# distance_df = pd.DataFrame(columns = [columns])

# for i, locx in enumerate(columns):
#     distance_df.loc[locx] = 0
#     for j,locy in enumerate(columns):
#         cord_x = (location_df.loc[i]['lat'], location_df.loc[i]['lon'])
#         cord_y = (location_df.loc[j]['lat'], location_df.loc[j]['lon'])
#         distance_df.loc[locx][locy] = distance.distance(cord_x, cord_y).kilometers

# distance_df.replace(0, np.nan, inplace=True)

### Locations and distances for combined government and kaiterra sensors

In [3]:
location_df = pd.read_excel("/scratch/ab9738/pollution_img/govdata/CombinedLocations.xlsx",engine='openpyxl')

In [4]:
location_df = location_df.drop(['Loc', 'Short Location', 'Address (as per lat-lon)'], axis=1)

In [5]:
# Adding UP Sensor Locations
# location_df.loc[len(location_df)] = ['SanjayNagar_UPPCP', '28.685382,77.453839']
# location_df.loc[len(location_df)] = ['Indirapuram_UPPCP', '28.646233,77.358075']
# location_df.loc[len(location_df)] = ['Vasundhara_UPPCP', '28.6603346,77.3572563']
# location_df.loc[len(location_df)] = ['Loni_UPPCP', '28.757294,77.278792']
# location_df.loc[len(location_df)] = ['NoidaSector62_IMD', '28.6245479,77.3577104']
# location_df.loc[len(location_df)] = ['NoidaSector116_UPPCP', '28.56912141,77.3939069']
# location_df.loc[len(location_df)] = ['KnowledgeParkV_UPPCP', '28.55856132,77.45445483']
# location_df.loc[len(location_df)] = ['KnowledgeParkIII_UPPCP', '28.47250249,77.48179193']
# location_df.loc[len(location_df)] = ['NoidaSector1_UPPCP', '28.58966084,77.30998866']
# location_df.loc[len(location_df)] = ['NoidaSector125_UPPCP', '28.54492244,77.32281108']

In [6]:
location_df

,Monitor ID,Lat-Lon
0,1FD7,"28.56502,77.19097"
1,2E9C,"28.523288,77.193379"
2,3ACF,"28.536967,77.203255"
3,4BE7,"28.5328704,77.2171122"
4,5D7A,"28.482569,77.100401"
5,8E2A,"28.4741026,77.1222129"
6,20CA,"28.574575,77.161071"
7,56C3,"28.543433,77.213555"
8,72CA,"28.455966,77.315964"
9,91B8,"28.50305,77.18566"


In [7]:
columns = [loc for loc in location_df['Monitor ID']]

In [8]:
distance_df = pd.DataFrame(columns = [columns])

In [9]:
for i, locx in enumerate(columns):
    distance_df.loc[locx] = 0
    for j,locy in enumerate(columns):
        cord_x = (float(location_df.loc[i]['Lat-Lon'].split(',')[0]),\
                  float(location_df.loc[i]['Lat-Lon'].split(',')[1]))
        cord_y = (float(location_df.loc[j]['Lat-Lon'].split(',')[0]),\
                  float(location_df.loc[j]['Lat-Lon'].split(',')[1]))
        distance_df.loc[locx][locy] = distance.distance(cord_x, cord_y).kilometers

In [10]:
distance_df.replace(0, np.nan, inplace=True)

In [11]:
distance_df

,1FD7,2E9C,3ACF,4BE7,5D7A,8E2A,20CA,56C3,72CA,91B8,...,Pusa_DPCC,Pusa_IMD,RKPuram_DPCC,Rohini_DPCC,Shadipur_CPCB,Sirifort_CPCB,SoniaVihar_DPCC,SriAurobindoMarg_DPCC,VivekVihar_DPCC,Wazirpur_DPCC
1FD7,NaN,4.631090,3.333396,4.386367,12.731477,12.117227,3.111082,3.257005,17.198875,6.887653,...,9.355496,9.355451,0.440088,19.821882,10.490474,2.930061,17.109382,3.732884,17.006421,15.143814
2E9C,4.631090,NaN,1.797964,2.554211,10.159921,8.846528,6.504179,2.980613,14.132738,2.366795,...,13.694747,13.694714,4.474881,24.277044,14.904803,3.730915,21.462711,0.947132,20.372039,19.751681
3ACF,3.333396,1.797964,NaN,1.430207,11.735812,10.558792,5.866121,1.236795,14.225369,4.134732,...,12.672032,12.671989,3.323050,23.155097,13.820706,1.940397,19.757641,1.425375,18.577282,18.420692
4BE7,4.386367,2.554211,1.430207,NaN,12.713154,11.346276,7.171915,1.221304,12.896397,4.516796,...,13.714005,13.713956,4.479391,24.082489,14.812708,1.948927,19.940403,2.643727,18.195896,19.177368
5D7A,12.731477,10.159921,11.735812,12.713154,NaN,2.332924,11.799929,12.969119,21.315502,8.650870,...,17.977563,17.977587,12.317694,27.768486,19.274408,13.581820,29.169086,10.316545,29.735608,24.901507
8E2A,12.117227,8.846528,10.558792,11.346276,2.332924,NaN,11.766824,11.789806,19.081439,6.991870,...,18.497117,18.497129,11.738028,28.641971,19.811065,12.479130,29.007984,9.192053,28.972885,25.368171
20CA,3.111082,6.504179,5.866121,7.171915,11.799929,11.766824,NaN,6.187574,20.067292,8.284218,...,7.355674,7.355654,2.824301,17.962254,8.628717,5.998720,17.369331,5.572620,18.567646,13.884499
56C3,3.257005,2.980613,1.236795,1.221304,12.969119,11.789806,6.187574,NaN,13.946158,5.242706,...,12.531223,12.531172,3.407929,22.869860,13.615267,0.809222,18.847245,2.653075,17.408676,17.956399
72CA,17.198875,14.132738,14.225369,12.896397,21.315502,19.081439,20.067292,13.946158,NaN,13.785535,...,26.271541,26.271479,17.347911,36.155570,27.237479,14.334904,28.950537,14.883608,23.980750,30.774427
91B8,6.887653,2.366795,4.134732,4.516796,8.650870,6.991870,8.284218,5.242706,13.785535,NaN,...,15.621679,15.621655,6.674352,26.232928,16.872535,6.029104,23.824766,3.166716,22.644162,21.894265


### Finding close-by sensors

In [12]:
def find_closeby(dist_df, radius):
    location_list = list(dist_df.index)
    num_locs = len(location_list)
    loc_pairs = []
    for i, locx in enumerate(location_list):
        for j in range(i+1, num_locs):
            if(dist_df.loc[locx][location_list[j]][0]<radius):
                loc_pairs.append((locx,location_list[j]))
    return loc_pairs

In [13]:
find_closeby(distance_df, 2.0)

[('1FD7', '97D7'),
 ('1FD7', 'BB4A'),
 ('1FD7', 'D804'),
 ('1FD7', 'E8E4'),
 ('1FD7', 'RKPuram_DPCC'),
 ('2E9C', '3ACF'),
 ('2E9C', 'EAC8'),
 ('2E9C', 'SriAurobindoMarg_DPCC'),
 ('3ACF', '4BE7'),
 ('3ACF', '56C3'),
 ('3ACF', 'E47A'),
 ('3ACF', 'EAC8'),
 ('3ACF', 'Sirifort_CPCB'),
 ('3ACF', 'SriAurobindoMarg_DPCC'),
 ('4BE7', '56C3'),
 ('4BE7', 'BFDC'),
 ('4BE7', 'E47A'),
 ('4BE7', 'Sirifort_CPCB'),
 ('5D7A', 'AyaNagar_IMD'),
 ('8E2A', 'AyaNagar_IMD'),
 ('56C3', 'A9BE'),
 ('56C3', 'D804'),
 ('56C3', 'E47A'),
 ('56C3', 'Sirifort_CPCB'),
 ('97D7', 'BB4A'),
 ('97D7', 'D804'),
 ('97D7', 'E8E4'),
 ('97D7', 'RKPuram_DPCC'),
 ('113E', '498F'),
 ('498F', 'A9BE'),
 ('498F', 'DF07'),
 ('498F', 'E1F8'),
 ('498F', 'NehruNagar_DPCC'),
 ('498F', 'Sirifort_CPCB'),
 ('603A', 'CBC7'),
 ('A9BE', 'D804'),
 ('A9BE', 'DF07'),
 ('A9BE', 'E1F8'),
 ('A9BE', 'Sirifort_CPCB'),
 ('A838', 'MDCNS_DPCC'),
 ('AshokVihar_DPCC', 'Wazirpur_DPCC'),
 ('BB4A', 'D804'),
 ('BB4A', 'E8E4'),
 ('BB4A', 'E47A'),
 ('BB4A', 'EAC8'

### Load the government and kaiterra data

In [14]:
govdata = pd.read_csv('/scratch/ab9738/pollution_img/govdata/govdata_1H_20180501_20201101.csv',\
                      index_col='monitor_id', usecols=['monitor_id','timestamp_round','pm10'])

In [15]:
govdata

,timestamp_round,pm10
monitor_id,,
AnandVihar_DPCC,2018-05-01 00:00:00+05:30,295.0
AnandVihar_DPCC,2018-05-01 01:00:00+05:30,202.0
AnandVihar_DPCC,2018-05-01 02:00:00+05:30,163.0
AnandVihar_DPCC,2018-05-01 03:00:00+05:30,140.0
AnandVihar_DPCC,2018-05-01 04:00:00+05:30,151.0
...,...,...
Wazirpur_DPCC,2020-10-31 19:00:00+05:30,377.0
Wazirpur_DPCC,2020-10-31 20:00:00+05:30,430.5
Wazirpur_DPCC,2020-10-31 21:00:00+05:30,468.5


In [16]:
kaidata = pd.read_csv('/scratch/ab9738/pollution_img/govdata/kaiterra_fieldeggid_1H_20180501_20201101.csv',\
                      index_col='field_egg_id', usecols=['field_egg_id','timestamp_round','pm10'])

In [17]:
kaidata = kaidata.dropna()

In [18]:
kaidata

,timestamp_round,pm10
field_egg_id,,
113E,2018-06-21 16:00:00+05:30,47.857143
113E,2018-06-21 17:00:00+05:30,40.250000
113E,2018-06-21 18:00:00+05:30,42.272727
113E,2018-06-21 19:00:00+05:30,37.833333
113E,2018-06-21 20:00:00+05:30,35.833333
...,...,...
EAC8,2020-07-11 02:00:00+05:30,7.714286
EAC8,2020-07-11 03:00:00+05:30,9.166667
EAC8,2020-07-11 04:00:00+05:30,7.800000


In [19]:
fulldata = pd.concat([govdata, kaidata])

In [20]:
fulldata = fulldata.loc[~(fulldata==0).all(axis=1)]
fulldata

,timestamp_round,pm10
AnandVihar_DPCC,2018-05-01 00:00:00+05:30,295.000000
AnandVihar_DPCC,2018-05-01 01:00:00+05:30,202.000000
AnandVihar_DPCC,2018-05-01 02:00:00+05:30,163.000000
AnandVihar_DPCC,2018-05-01 03:00:00+05:30,140.000000
AnandVihar_DPCC,2018-05-01 04:00:00+05:30,151.000000
...,...,...
EAC8,2020-07-11 02:00:00+05:30,7.714286
EAC8,2020-07-11 03:00:00+05:30,9.166667
EAC8,2020-07-11 04:00:00+05:30,7.800000
EAC8,2020-07-11 05:00:00+05:30,7.666667


### Estimate the sensor substitution error

In [21]:
pairs = find_closeby(distance_df, 2.0)

In [22]:
pairs

[('1FD7', '97D7'),
 ('1FD7', 'BB4A'),
 ('1FD7', 'D804'),
 ('1FD7', 'E8E4'),
 ('1FD7', 'RKPuram_DPCC'),
 ('2E9C', '3ACF'),
 ('2E9C', 'EAC8'),
 ('2E9C', 'SriAurobindoMarg_DPCC'),
 ('3ACF', '4BE7'),
 ('3ACF', '56C3'),
 ('3ACF', 'E47A'),
 ('3ACF', 'EAC8'),
 ('3ACF', 'Sirifort_CPCB'),
 ('3ACF', 'SriAurobindoMarg_DPCC'),
 ('4BE7', '56C3'),
 ('4BE7', 'BFDC'),
 ('4BE7', 'E47A'),
 ('4BE7', 'Sirifort_CPCB'),
 ('5D7A', 'AyaNagar_IMD'),
 ('8E2A', 'AyaNagar_IMD'),
 ('56C3', 'A9BE'),
 ('56C3', 'D804'),
 ('56C3', 'E47A'),
 ('56C3', 'Sirifort_CPCB'),
 ('97D7', 'BB4A'),
 ('97D7', 'D804'),
 ('97D7', 'E8E4'),
 ('97D7', 'RKPuram_DPCC'),
 ('113E', '498F'),
 ('498F', 'A9BE'),
 ('498F', 'DF07'),
 ('498F', 'E1F8'),
 ('498F', 'NehruNagar_DPCC'),
 ('498F', 'Sirifort_CPCB'),
 ('603A', 'CBC7'),
 ('A9BE', 'D804'),
 ('A9BE', 'DF07'),
 ('A9BE', 'E1F8'),
 ('A9BE', 'Sirifort_CPCB'),
 ('A838', 'MDCNS_DPCC'),
 ('AshokVihar_DPCC', 'Wazirpur_DPCC'),
 ('BB4A', 'D804'),
 ('BB4A', 'E8E4'),
 ('BB4A', 'E47A'),
 ('BB4A', 'EAC8'

In [23]:
def mean_rel_abs_error(df1, df2):
    assert(df1.index == df2.index).all()
    if(len(list(df1.index))):
        total_rel_err = 0
        total_valid_len = 0
        for ind in list(df1.index):
            rel_err = abs(df2.loc[ind][0]-df1.loc[ind][0])/df1.loc[ind][0]
            if(not math.isnan(rel_err)):
                total_rel_err += rel_err
                total_valid_len += 1
        mean_err = total_rel_err/total_valid_len
        return mean_err

In [24]:
def mean_abs_error(df1, df2):
    assert(df1.index == df2.index).all()
    if(len(list(df1.index))):
        total_rel_err = 0
        total_valid_len = 0
        for ind in list(df1.index):
            rel_err = abs(df2.loc[ind][0]-df1.loc[ind][0])
            if(not math.isnan(rel_err)):
                total_rel_err += rel_err
                total_valid_len += 1
        mean_err = total_rel_err/total_valid_len
        return mean_err

In [25]:
mean_rel_sensubs_err = []
mean_sensubs_err = []

In [26]:
for indv_pair in pairs:
    df1 = fulldata.loc[indv_pair[0]].set_index('timestamp_round')
    df2 = fulldata.loc[indv_pair[1]].set_index('timestamp_round')
    common_timestamps = list(set(list(df1.index)).intersection(list(df2.index)))
    mean_rel_sensubs_err.append(mean_rel_abs_error(df1.loc[common_timestamps], df2.loc[common_timestamps]))
    mean_rel_sensubs_err.append(mean_rel_abs_error(df2.loc[common_timestamps], df1.loc[common_timestamps]))

<ipython-input-23-34649d0da35e>:7: RuntimeWarning: divide by zero encountered in double_scalars
  rel_err = abs(df2.loc[ind][0]-df1.loc[ind][0])/df1.loc[ind][0]


In [27]:
mean_rel_sensubs_err

[0.3753343733646914,
 0.44402791884826837,
 8.59490953572248,
 0.8850577374159948,
 0.31933598401843655,
 0.25611625971303886,
 0.37918716441339095,
 0.2232973962690909,
 1.97193783238935,
 inf,
 0.354134478595232,
 0.8713927584255132,
 0.4300201467624039,
 0.4125821945444711,
 0.5597057016611673,
 0.4165334542235097,
 inf,
 0.13339712974257614,
 inf,
 0.38542197508888654,
 None,
 None,
 inf,
 0.24770635490259302,
 inf,
 0.5278515283997647,
 inf,
 0.38480897564045063,
 0.2764574816212266,
 0.44892013703710865,
 0.12575313762383356,
 0.12800785154700034,
 10.482246578360675,
 0.902494984400742,
 2.6444434223624453,
 0.5338066556764451,
 1.6232673610019968,
 0.5549390576914804,
 0.8091264605332603,
 9.83706874710883,
 0.17409540393807746,
 0.19923281005916535,
 0.16200293815218483,
 0.16067770284424876,
 None,
 None,
 3.9008108797708285,
 0.5879952469766944,
 10.308824922649807,
 0.9666930261068705,
 0.27238723462385134,
 0.19959914597349376,
 0.5070585084414174,
 0.30752974386862,
 inf,

In [28]:
for indv_pair in pairs:
    df1 = fulldata.loc[indv_pair[0]].set_index('timestamp_round')
    df2 = fulldata.loc[indv_pair[1]].set_index('timestamp_round')
    common_timestamps = list(set(list(df1.index)).intersection(list(df2.index)))
    mean_sensubs_err.append(mean_abs_error(df1.loc[common_timestamps], df2.loc[common_timestamps]))

In [39]:
for i in range(len(pairs)):
    print(pairs[i], distance_df.loc[pairs[i][0]][pairs[i][1]][0], mean_sensubs_err[i])

('1FD7', '97D7') 0.6037274927539169 22.69545679972793
('1FD7', 'BB4A') 1.163083412864811 672.429102736244
('1FD7', 'D804') 1.883834833905887 19.84435375776403
('1FD7', 'E8E4') 0.6422720265903663 43.27472427558421
('1FD7', 'RKPuram_DPCC') 0.44008753476845675 114.81184725163874
('2E9C', '3ACF') 1.7979637593342657 27.00334209566731
('2E9C', 'EAC8') 1.909522848757657 53.06211938384592
('2E9C', 'SriAurobindoMarg_DPCC') 0.9471321639241662 64.04445865515957
('3ACF', '4BE7') 1.4302072247115156 12.607270027187079
('3ACF', '56C3') 1.2367949753672771 51.23374413285128
('3ACF', 'E47A') 0.7231735328327823 None
('3ACF', 'EAC8') 1.6451821832133084 19.43399910941691
('3ACF', 'Sirifort_CPCB') 1.940396590468067 69.39997682149546
('3ACF', 'SriAurobindoMarg_DPCC') 1.425374799178763 29.635104263179652
('4BE7', '56C3') 1.2213043497456841 34.339144677976776
('4BE7', 'BFDC') 1.336381791440725 11.405188774188769
('4BE7', 'E47A') 1.4420859583222487 909.762253050212
('4BE7', 'Sirifort_CPCB') 1.9489268466531544 1

In [38]:
distance_df.loc[pairs[0][0]][pairs[0][1]][0]

0.6037274927539169

### Calculate Image Distances

In [134]:
image_list = os.listdir('/scratch/ab9738/pollution_img/new_images/')

In [135]:
img_latlong_list = []

In [136]:
for image in image_list:
    latlong_str = image.split('___')[1]
    latlong_tuple = tuple(latlong_str.split('_'))
    img_latlong_list.append(latlong_tuple)

In [138]:
img_latlong_list = [(float(lat), float(long)) for lat, long in img_latlong_list]

In [139]:
img_latlong_list

[(28.661858, 77.314443),
 (28.664578, 77.393921),
 (28.475996, 77.483764),
 (28.630176, 77.326735),
 (28.635686, 77.393773),
 (28.540366, 77.33232),
 (28.631465, 77.364205),
 (28.649286, 77.379147),
 (28.660801, 77.377701),
 (28.688277, 77.451112),
 (28.585753, 77.310184),
 (28.628635, 77.364955),
 (28.552655, 77.105907),
 (28.63739, 77.314455),
 (28.569345, 77.216624),
 (28.695991, 77.441333),
 (28.644784, 77.313864),
 (28.648623, 77.304227),
 (28.648374, 77.303762),
 (28.58925, 77.154483),
 (28.474239, 77.485458),
 (28.661537, 77.464322),
 (28.644775, 77.314376),
 (28.633781, 77.282711),
 (28.667866, 77.314722),
 (28.64911, 77.318156),
 (28.584914, 77.230163),
 (28.647227, 77.379171),
 (28.605417, 77.226652),
 (28.531033, 77.362267),
 (28.636282, 77.443121),
 (28.662044, 77.314296),
 (28.635229, 77.45851),
 (28.629601, 77.27662),
 (28.627126, 77.317232),
 (28.688788, 77.45196),
 (28.691533, 77.438049),
 (28.605513, 77.289196),
 (28.631952, 77.370305),
 (28.596983, 77.292413),
 (28.62

In [167]:
columns = [loc for loc in location_df['Monitor ID']]
imgdist_df = pd.DataFrame(columns = [columns])

In [168]:
for i, cord_x in enumerate(img_latlong_list):
    imgdist_df.loc[i] = 0
    for j,locy in enumerate(columns):
        cord_y = (float(location_df.loc[j]['Lat-Lon'].split(',')[0]),\
                  float(location_df.loc[j]['Lat-Lon'].split(',')[1]))
        imgdist_df.loc[i][locy] = distance.distance(cord_x, cord_y).kilometers

In [169]:
img_min_dist = imgdist_df.min(axis=1)

In [179]:
img_min_dist[img_min_dist<1]

2      0.432667
6      0.995506
9      0.417105
10     0.433523
11     0.840865
         ...   
490    0.840865
491    0.285337
497    0.754610
498    0.811211
503    0.468267
Length: 167, dtype: float64